In [1]:
#Dependencies
import pandas as pd
import numpy as np
import plotly.plotly as py
from patsy import dmatrices
import statsmodels.api as sm
import matplotlib.pyplot as plt

#CSV File to load
file_to_load = "./Data_dc-residential-properties/DC_Properties.csv"
#print ('goodpath')

#Read file and store into Pandas DF. Scatter plots with price and 
real_estate_data_pd = pd.read_csv(file_to_load)
real_estate_data_pd.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning:

Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.



,Unnamed: 0,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,...,LONGITUDE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT
0,0,4,0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,...,-77.040832,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
1,1,3,1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,...,-77.040764,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
2,2,3,1,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,...,-77.040678,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
3,3,3,1,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,...,-77.040629,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
4,4,2,1,Warm Cool,Y,1.0,11,3,1913.0,2012.0,...,-77.039361,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW


In [2]:
#Clean DF to remove all unrelated variables
isolated_pd = real_estate_data_pd.drop(columns=["Unnamed: 0", "NUM_UNITS", "ROOMS",
                                                "AYB", "YR_RMDL", "EYB","CITY","STATE", "STORIES", "SALE_NUM", 
                                                "BLDG_NUM", "GRADE", "CNDTN", "EXTWALL", "ROOF", "INTWALL",
                                                "KITCHENS", "FIREPLACES", "USECODE", "GIS_LAST_MOD_DTTM",
                                                "CMPLX_NUM", "ASSESSMENT_NBHD", "ASSESSMENT_SUBNBHD", 
                                                "CENSUS_TRACT", "CENSUS_BLOCK", "QUADRANT", "X", "Y"])
isolated_pd.head()

,BATHRM,HF_BATHRM,HEAT,AC,BEDRM,SALEDATE,PRICE,QUALIFIED,GBA,STYLE,...,LANDAREA,SOURCE,LIVING_GBA,FULLADDRESS,ZIPCODE,NATIONALGRID,LATITUDE,LONGITUDE,WARD,SQUARE
0,4,0,Warm Cool,Y,4,2003-11-25 00:00:00,1095000.0,Q,2522.0,3 Story,...,1680,Residential,NaN,1748 SWANN STREET NW,20009.0,18S UJ 23061 09289,38.914680,-77.040832,Ward 2,152
1,3,1,Warm Cool,Y,5,2000-08-17 00:00:00,NaN,U,2567.0,3 Story,...,1680,Residential,NaN,1746 SWANN STREET NW,20009.0,18S UJ 23067 09289,38.914683,-77.040764,Ward 2,152
2,3,1,Hot Water Rad,Y,5,2016-06-21 00:00:00,2100000.0,Q,2522.0,3 Story,...,1680,Residential,NaN,1744 SWANN STREET NW,20009.0,18S UJ 23074 09289,38.914684,-77.040678,Ward 2,152
3,3,1,Hot Water Rad,Y,5,2006-07-12 00:00:00,1602000.0,Q,2484.0,3 Story,...,1680,Residential,NaN,1742 SWANN STREET NW,20009.0,18S UJ 23078 09288,38.914683,-77.040629,Ward 2,152
4,2,1,Warm Cool,Y,3,NaN,NaN,U,5255.0,3 Story,...,2032,Residential,NaN,1804 NEW HAMPSHIRE AVENUE NW,20009.0,18S UJ 23188 09253,38.914383,-77.039361,Ward 2,152


In [3]:
final_df=isolated_pd.loc[isolated_pd['QUALIFIED']=='Q']

In [4]:
final=final_df.fillna(value={'GBA':0.0,'LIVING_GBA':0.0})
final["Combined_GBA"]=final["GBA"]+final['LIVING_GBA']

In [12]:
heatindicators=pd.get_dummies(final["HEAT"])
heatindicators.mean()
zipcode_ind=pd.get_dummies(final["ZIPCODE"])
zipcode_ind.mean()
ward_ind=pd.get_dummies(final["WARD"])
ward_ind.mean()
final['WARD'].value_counts()
struct_ind=pd.get_dummies(final['STRUCT'], columns=['default','multi','row_end','row_inside','semi_detached','single','town_end','town_inside'])
struct_ind.head()
ac_ind=pd.get_dummies(final["AC"])
res_ind=pd.get_dummies(final["SOURCE"])
dummies=pd.DataFrame({
    'Y': ac_ind.Y,
    'ward_1': ward_ind['Ward 1'],
    'ward_2': ward_ind['Ward 2'],
    'ward_3': ward_ind['Ward 3'],
    'ward_4': ward_ind['Ward 4'],
    'ward_5': ward_ind['Ward 5'],
    'ward_6': ward_ind['Ward 6'],
    'ward_7': ward_ind['Ward 7'],
    'ward_8': ward_ind['Ward 8'],
    'default':struct_ind['Default'],
    'multi':struct_ind['Multi'],
    'row_end':struct_ind['Row End'],
    'row_inside':struct_ind['Row Inside'],
    'semi_detached':struct_ind['Semi-Detached'],
    'single':struct_ind['Single'],
    'town_end':struct_ind['Town End'],
    'town_inside':struct_ind['Town Inside'],
    'log_GBA_RES_IND':np.log(final['Combined_GBA'])*res_ind.Residential,
    'GBA_RES_IND':final['Combined_GBA']*res_ind.Residential,
    'Residential':res_ind.Residential,
    'log_price':np.log(final['PRICE']),
    'log_Combined_GBA':np.log(final['Combined_GBA']),
    'priceinhunthou':final['PRICE']/100000
})
res_ind.head()
ac_ind=pd.get_dummies(final['AC'])
#ac_ind.mean()
really_final=pd.concat([final,dummies],sort=False,axis=1)
really_final
#col_names=list(final)


,BATHRM,HF_BATHRM,HEAT,AC,BEDRM,SALEDATE,PRICE,QUALIFIED,GBA,STYLE,...,semi_detached,single,town_end,town_inside,log_GBA_RES_IND,GBA_RES_IND,Residential,log_price,log_Combined_GBA,priceinhunthou
0,4,0,Warm Cool,Y,4,2003-11-25 00:00:00,1095000.0,Q,2522.0,3 Story,...,0,0,0,0,7.832808,2522.0,1,13.906265,7.832808,10.95000
2,3,1,Hot Water Rad,Y,5,2016-06-21 00:00:00,2100000.0,Q,2522.0,3 Story,...,0,0,0,0,7.832808,2522.0,1,14.557448,7.832808,21.00000
3,3,1,Hot Water Rad,Y,5,2006-07-12 00:00:00,1602000.0,Q,2484.0,3 Story,...,0,0,0,0,7.817625,2484.0,1,14.286763,7.817625,16.02000
5,3,2,Hot Water Rad,Y,5,2010-02-26 00:00:00,1950000.0,Q,5344.0,4 Story,...,0,0,0,0,8.583730,5344.0,1,14.483340,8.583730,19.50000
7,3,1,Hot Water Rad,Y,4,2011-09-29 00:00:00,1050000.0,Q,2401.0,3 Story,...,0,0,0,0,7.783641,2401.0,1,13.864301,7.783641,10.50000
8,3,1,Warm Cool,Y,3,2018-05-03 00:00:00,1430000.0,Q,1488.0,2 Story,...,0,0,0,0,7.305188,1488.0,1,14.173185,7.305188,14.30000
14,3,1,Warm Cool,Y,3,2011-09-30 00:00:00,1325000.0,Q,2692.0,2 Story,...,0,0,0,0,7.898040,2692.0,1,14.096923,7.898040,13.25000
16,3,1,Warm Cool,Y,3,2010-11-04 00:00:00,1240000.0,Q,1662.0,2 Story,...,0,0,0,0,7.415777,1662.0,1,14.030622,7.415777,12.40000
19,3,1,Hot Water Rad,Y,3,2001-07-23 00:00:00,592250.0,Q,1598.0,2 Story,...,0,0,0,0,7.376508,1598.0,1,13.291684,7.376508,5.92250
22,1,0,Forced Air,Y,3,2006-01-09 00:00:00,907400.0,Q,1790.0,2 Story,...,1,0,0,0,7.489971,1790.0,1,13.718339,7.489971,9.07400


In [16]:
In [11]: y, X = dmatrices('priceinhunthou ~ Combined_GBA + GBA_RES_IND + Residential + Y + ward_8 + ward_2 + ward_3 + ward_4 + ward_5 + ward_6 + ward_7', data=really_final, return_type='dataframe')


In [24]:
mod = sm.OLS(y, X)
res = mod.fit(cov_type='HC1')


In [25]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:         priceinhunthou   R-squared:                       0.560
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     3559.
Date:                Fri, 26 Apr 2019   Prob (F-statistic):               0.00
Time:                        15:29:14   Log-Likelihood:            -1.9933e+05
No. Observations:               76233   AIC:                         3.987e+05
Df Residuals:                   76221   BIC:                         3.988e+05
Df Model:                          11                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.3816      0.114    -20.900   

In [21]:
final.PRICE.describe()

count    7.623300e+04
mean     5.453246e+05
std      4.985197e+05
min      2.500000e+02
25%      2.685000e+05
50%      4.250000e+05
75%      6.825000e+05
max      2.200000e+07
Name: PRICE, dtype: float64